# Test

In [144]:
import weaviate
import json
from llama_index import download_loader
from pathlib import Path
from llama_index import download_loader, VectorStoreIndex, ServiceContext, SimpleDirectoryReader, Document
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index.utils import globals_helper

# PandasExcelReader = download_loader("PandasExcelReader")

# loader = PandasExcelReader(pandas_config={"header": 0})
# documents = loader.load_data(file=Path('.\metric\GRI Benchmark metrics.xlsx'))


auth_config = weaviate.AuthApiKey(api_key="1DUpoOCR1eX68sQkDIk1SijEW1obWnIMbWeH")  # Replace w/ your Weaviate instance API key

# Instantiate the client with the auth config
client = weaviate.Client(
    url="https://gri-esg-k5ypq3jp.weaviate.network",  # Replace w/ your endpoint
    auth_client_secret=auth_config
)

some_objects = client.data_object.get()

In [145]:
some_objects

{'deprecations': None,
 'objects': [{'class': 'CustomCollection',
   'creationTimeUnix': 1697679853347,
   'id': '00d68ffa-b8b1-410a-9ecc-a7ef9a4f3111',
   'lastUpdateTimeUnix': 1697679853347,
   'properties': {'chunk_id': 2,
    'content': 'to 2023 compared to 2018, with at least $25 \nmillion like-for-like OPEX savings in 2023\n fSmart, targeted OPEX optimization approaches include \nexamining warehousing costs, reviewing operating models for \nretail sites, outsourcing, and vendor negotiations, simplifying \nmarketing expenditures and back-office procedures, as well as \noptimizing utility usage\n f Accelerating expansion CAPEX and improving network efficiency\n fComplete store network renovation with a new look and feel\n fOptimize product category management\n fImprove fresh food and premium coffee offerings to boost \nbasket size and margins.\n fIncrease store conversion rate (fuel to non-fuel conversion)\n fexpand into e-commerce and franchisee models fExpand more effectively in

In [184]:

query = (
        client.query.get("MetricVector", ["index","level1","level2","level3","related_doc","comparison"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_limit(1)
    )
metric_vector = query.do()

query = (
        client.query.get("CustomCollection", ["content"])
        # Optionally retrieve the vector embedding by adding `vector` to the _additional fields
        .with_limit(2)
    )
pdf_vector = query.do()

In [187]:
metric_vector = metric_vector['data']['Get']
pdf_vector = pdf_vector['data']['Get']

In [188]:
metric_vector = metric_vector['data']['Get']
pdf_vector = pdf_vector['data']['Get']
merged_dict = {**metric_vector, **pdf_vector}

merged_dict

{'MetricVector': [{'comparison': 'Aligned. The report provides details of the governance structure and stakeholder engagement.',
   'index': 34,
   'level1': 'Governance',
   'level2': 'Stakeholder Engagement',
   'level3': 'Identifying and selecting stakeholders',
   'related_doc': '[{"_additional": {"certainty": 0.8211186826229095}, "content": "Authorities. \\nADNOC Distribution employs different engagement \\nmethods for each stakeholder Group to identify \\ncritical concerns raised or their topics of interest and \\nengage with them to address those concerns through \\ndiffering mechanisms.ADNOC DISTRIBUTION \\nCONTINUES TO \\nPRIORITIZE ITS \\nLONG-TERM \\nSUSTAINABILITY \\nSUCCESSES \\nTHROUGH THE LENS \\nOF CREATING VALUE \\nFOR ITS ESTEEMED \\nSTAKEHOLDERS.\\n8. 2-29 Approach to stakeholder engagement\\n29 \\nEnvironmental, Social & Governance Report 2022 28 \\nEnvironmental, Social & Governance Report 2022"}, {"_additional": {"certainty": 0.7903783321380615}, "content": "stake

In [195]:
response = {}

# 在指定键的值中添加新值
response['result'] = 'result2'
response['question'] = 'question3'

{'question': (user_question)}
# 打印更新后的字典
print(response)


{'result': 'resultresult1result2', 'question': 'questionquestion1question3'}


In [203]:
msg = {'user': 'Please give me a summary about Environment level of this company', 'role': 'user'}
print(msg['user'])

Please give me a summary about Environment level of this company


In [198]:
for i, message in enumerate(reversed(response)):
    if i % 2 == 0:
        print(message.get(contents))

    else:
        print(message)


AttributeError: 'str' object has no attribute 'get'

In [161]:
chunk_obj_list = list(metric_vector['data']['Get']['MetricVector'])

In [175]:
import re

df = pd.DataFrame(chunk_obj_list)
compared_df = df.sort_values(by='index')


# compared_df = compare_matric(df)
not_align_count = (compared_df['comparison'].str.lower().str.contains('not aligned')).sum()
# align_count = (compared_df['comparison'].str.lower().str.contains('aligned')).sum()
align_count = len(compared_df) - not_align_count

# 根据dataframe生成答案

In [178]:
import requests
class Bam:

    def call_bam(
        self,
        input_str: str,
        task: str = "qa",
    ) -> dict:
        """Call bam RESTAPI.

        Args:
            input_str (str): The input text to the model.
            task (str): The task to perform, such as qa, summarization.
            
        Returns:
            (dict): The bam output.
            
        Raises:
            requests.RequestException: If there is an error making the API request.
            ValueError: If the url and bam_key are not provided.

        """
        try:
            url = 'https://bam-api.res.ibm.com/v1/generate'
        except:
            raise ValueError("url must be provided or set in config file.")
        try:
            bam_key = 'pak-KC4mPMsWZKPjmp0U1kkjErhxX_XlH8l9k0C3nhHSSg4'
        except:
            raise ValueError("BAM_KEY must be provided or set in ENV.")


        bam_input = {
            "model_id": 'google/flan-ul2',
            "inputs": [input_str],
            "parameters": {
                "decoding_method": 'sample',
                "temperature": 0.1,
                "top_p": 0.95,
                "top_k": 50,
                "repetition_penalty": 1,
                "min_new_tokens":0,
                "max_new_tokens": 100
            }
        }


        try:
            headers = {
                "Content-Type": "application/json",
                "Authorization": f"Bearer {bam_key}"
            }
            response = requests.post(
                url,
                headers=headers,
                data=json.dumps(bam_input)
            )
            response.raise_for_status()
            result = response.json()
            return result
        except requests.exceptions.RequestException as e:
            return {}


In [180]:
def question_answer(model_platform: str, context: str, question:str, task:str) -> dict:
    """Get question answer from LLM.

    Args:
        model_platform (str): BAM or WATSONX.
        context (str): the related text.
        question (str): the question.

    Returns:
        result (dict): the model response.
    """    
    if task == 'qa_pdf':
        prompt = f'''
                You are a academic assistant AI chatbot here to assist the user based on the academic PDFs they uploaded, and the subsequent openAI embeddings. 
                This academic persona allows you to use as much outside academic responses as you can.
                But remember this is an app for academix PDF question. Please respond in as academic a way as possible, with an academix audience in mind.
                Please assist the user to the best of your knowledge, with this academic persona based on the following PDF context, embeddings and the user input. 
                
                PDF CONTEXT:
                {context}
                USER INPUT: 
                {question}
                RESPONSE:
        '''
    elif task == 'qa':
        prompt = '''
                You are a assistant AI chatbot based on your knowledge base. If you don't know just say I don't know.
                USER INPUT: 
                {question}
                RESPONSE:
        '''

    result = {}
    if model_platform=="BAM" and task=="qa_pdf":
        bam_instance = Bam()
        result = bam_instance.call_bam(input_str=pdf_prompt,task="qa_pdf")
        
    elif model_platform=="BAM" and task=="qa":
        bam_instance = Bam()
        result = bam_instance.call_bam(input_str=gen_prompt,task="qa")
    logger.info(f"{model_platform} QA result: {str(result)}")
    return result

In [107]:
def handle_userinput(user_question):
    if user_question:
        question_answer(model_platform="BAM", context="1", question=user_question, task='qa_pdf')
        # response = st.session_state.conversation({'question': (prompt+user_question)})
        # st.session_state.chat_history = response['chat_history']
        with st.spinner('Generating response...'):
            display_convo(user_question)
    else:
        question_answer(model_platform="BAM", context="1", question=user_question, task='qa')
        with st.spinner('Generating response...'):
            display_convo(user_question)

c:\Users\ZZ0CTQ672\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\dtypes\cast.py:1841: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
